<a href="https://colab.research.google.com/github/souravb65/TensorFlow-in-Practice-Specialization./blob/master/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [0]:
tf.enable_eager_execution()

In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

W0708 14:10:06.319908 139701857953664 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np


In [0]:
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []


In [0]:
for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

In [0]:
for s, l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
print(word_index)

{'<OOV>': 1, 'the': 2, 'and': 3, 'a': 4, 'of': 5, 'to': 6, 'is': 7, 'br': 8, 'in': 9, 'it': 10, 'i': 11, 'this': 12, 'that': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'movie': 18, 'but': 19, 'film': 20, "'s": 21, 'on': 22, 'you': 23, 'not': 24, 'are': 25, 'his': 26, 'he': 27, 'have': 28, 'be': 29, 'one': 30, 'all': 31, 'at': 32, 'by': 33, 'they': 34, 'an': 35, 'who': 36, 'so': 37, 'from': 38, 'like': 39, 'her': 40, "'t": 41, 'or': 42, 'just': 43, 'there': 44, 'about': 45, 'out': 46, "'": 47, 'has': 48, 'if': 49, 'some': 50, 'what': 51, 'good': 52, 'more': 53, 'very': 54, 'when': 55, 'she': 56, 'up': 57, 'can': 58, 'b': 59, 'time': 60, 'no': 61, 'even': 62, 'my': 63, 'would': 64, 'which': 65, 'story': 66, 'only': 67, 'really': 68, 'see': 69, 'their': 70, 'had': 71, 'were': 72, 'me': 73, 'well': 74, 'we': 75, 'than': 76, 'much': 77, 'been': 78, 'get': 79, 'bad': 80, 'will': 81, 'people': 82, 'do': 83, 'also': 84, 'other': 85, 'into': 86, 'because': 87, 'great': 88, 'first': 89, 'hi

In [0]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [16]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
  
print(decode_review(padded[1]))
print(training_sentences[1])

b'this was a wonderfully clever and entertaining movie that i shall never tire of watching many many times the casting was magnificent in matching up the young with the older characters there are those of us out here who really do appreciate good actors and an intelligent story format as for <OOV> <OOV> she is beautiful and a gift to any kind of production in which she stars i always make a point to see <OOV> <OOV> in all her performances she is a superb actress and a pleasure to watch as each transformation of her character comes to life i can only be grateful when i see such an outstanding picture for most of the motion pictures made more
b'This was a wonderfully clever and entertaining movie that I shall never tire of watching many, many times. The casting was magnificent in matching up the young with the older characters. There are those of us out here who really do appreciate good actors and an intelligent story format. As for Judi Dench, she is beautiful and a gift to any kind of

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [18]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

W0708 14:21:58.591843 139701857953664 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 227us/sample - loss: 0.4858 - acc: 0.7528 - val_loss: 0.3433 - val_acc: 0.8511
Epoch 2/10
25000/25000 [==============================] - 5s 198us/sample - loss: 0.2429 - acc: 0.9055 - val_loss: 0.3749 - val_acc: 0.8383
Epoch 3/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0988 - acc: 0.9734 - val_loss: 0.4509 - val_acc: 0.8273
Epoch 4/10
25000/25000 [==============================] - 5s 192us/sample - loss: 0.0257 - acc: 0.9964 - val_loss: 0.5332 - val_acc: 0.8245
Epoch 5/10
25000/25000 [==============================] - 5s 215us/sample - loss: 0.0076 - acc: 0.9992 - val_loss: 0.6037 - val_acc: 0.8233
Epoch 6/10
25000/25000 [==============================] - 6s 243us/sample - loss: 0.0026 - acc: 0.9998 - val_loss: 0.6593 - val_acc: 0.8248
Epoch 7/10
25000/25000 [==============================] - 5s 193us/sample - loss: 9.7786e-04 - acc: 1.0000 - v

In [19]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [22]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1537], [1537], [4738], [], [790], [2016], [11], [2921], [2191], [], [790], [2016], [11], [579], [], [11], [579], [], [4], [1783], [4], [4507], [11], [2921], [1277], [], [], [2016], [1005], [2921], [968], [579], [790], []]
